In [27]:
import pandas as pd
import openai
import llama_index

import pandas as pd
from dotenv import load_dotenv
import os
from llama_index.core import Document
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.mistralai import MistralAIEmbedding
from sentence_transformers import SentenceTransformer

from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from IPython.display import Markdown, display
import chromadb
from chromadb.config import Settings

In [28]:
import os
from dotenv import load_dotenv
import openai

# Load environment variables from .env file
load_dotenv(dotenv_path='../.env')

# Accessing the API key from environment variables
openai.api_key = os.getenv("OPENAI_API_KEY")

chroma_server_host = os.getenv("CHROMA_SERVER_HOST")
chroma_server_http_port = os.getenv("CHROMA_SERVER_HTTP_PORT")

In [29]:
# create client and a new collection
chroma_client = chromadb.HttpClient(host=chroma_server_host, port=chroma_server_http_port)
chroma_collection = chroma_client.create_collection("LumadAI_collection")
documents = SimpleDirectoryReader('./data_final').load_data()

2024-04-16 23:10:30,291 ERROR chromadb.telemetry.product.posthog: Failed to send telemetry event ClientStartEvent: You are using a deprecated configuration of Chroma.

If you do not have data you wish to migrate, you only need to change how you construct
your Chroma client. Please see the "New Clients" section of https://docs.trychroma.com/migration.
________________________________________________________________________________________________

If you do have data you wish to migrate, we have a migration tool you can use in order to
migrate your data to the new Chroma architecture.
Please `pip install chroma-migrate` and run `chroma-migrate` to migrate your data and then
change how you construct your Chroma client.

See https://docs.trychroma.com/migration for more information or join our discord at https://discord.gg/8g5FESbj for help!


In [30]:
embed_model = OpenAIEmbedding(model="text-embedding-3-small")

In [31]:
# save to disk

db = chromadb.HttpClient(host=chroma_server_host, port=chroma_server_http_port)
chroma_collection = db.get_or_create_collection("LumadAI_collection")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model
)

2024-04-16 23:10:31,491 ERROR chromadb.telemetry.product.posthog: Failed to send telemetry event ClientStartEvent: You are using a deprecated configuration of Chroma.

If you do not have data you wish to migrate, you only need to change how you construct
your Chroma client. Please see the "New Clients" section of https://docs.trychroma.com/migration.
________________________________________________________________________________________________

If you do have data you wish to migrate, we have a migration tool you can use in order to
migrate your data to the new Chroma architecture.
Please `pip install chroma-migrate` and run `chroma-migrate` to migrate your data and then
change how you construct your Chroma client.

See https://docs.trychroma.com/migration for more information or join our discord at https://discord.gg/8g5FESbj for help!


In [32]:
# load from disk
db2 = chromadb.HttpClient(host=chroma_server_host, port=chroma_server_http_port)
chroma_collection = db2.get_or_create_collection("LumadAI_collection")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model=embed_model,
)

2024-04-16 23:18:28,436 ERROR chromadb.telemetry.product.posthog: Failed to send telemetry event ClientStartEvent: You are using a deprecated configuration of Chroma.

If you do not have data you wish to migrate, you only need to change how you construct
your Chroma client. Please see the "New Clients" section of https://docs.trychroma.com/migration.
________________________________________________________________________________________________

If you do have data you wish to migrate, we have a migration tool you can use in order to
migrate your data to the new Chroma architecture.
Please `pip install chroma-migrate` and run `chroma-migrate` to migrate your data and then
change how you construct your Chroma client.

See https://docs.trychroma.com/migration for more information or join our discord at https://discord.gg/8g5FESbj for help!


In [33]:
# Query Data from the Chroma Docker index
query_engine = index.as_query_engine()
response = query_engine.query("How to get a CADT?")
display(Markdown(f"<b>{response}</b>"))

<b>To obtain a Certificate of Ancestral Domain Title (CADT), indigenous peoples must follow the procedures outlined in the Indigenous Peoples Rights Act (IPRA) of 1997. This involves submitting an application to the National Commission on Indigenous Peoples (NCIP), providing necessary documentation, undergoing a process of delineation and recognition of ancestral domains, and securing the approval of the CADT by the NCIP.</b>